# 0. 데이터 전처리
---
1차 목표: pdf로 작성된 자기소개서 파일 텍스트화하여 excel 파일로 저장 (220824 완료)

-> 보완사항 총 73개 항목 중 52개 데이터만 처리됨.(파일 호환 문제로 판단)

-> docx 파일 추가 예정

2차 목표: 자기소개서 질문 내용 데이터에서 배제 필요, 정규화 필요
3차 목표: 학교 유형별 label 추가

In [1]:
import glob                  # 해당 디렉토리의 파일들을 탐색하기 위한 모듈
from io import StringIO      # StringIO는 파일을 흉내내는 객체로, 문자열 데이터를 임시로 처리할 때 활용
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

import re
import pandas as pd
import numpy as np

In [2]:
def read_pdf(file_path, convertStartNo = 1):
    '''파일 경로와 변환 시작 페이지를 매개변수로 하여 해당 PDF 파일을 text로 반환. 
       file_path, converStartNo = 1이 기본값'''
    output_String = StringIO()
    with open(file_path, 'rb') as file: 
        # rb는 파일을 byte 형식으로 읽어옴
        # 아래 코드들은 pdf를 byte 형식에서 다시 한글 문서로 변환하는 과정으로 추정 
        parser = PDFParser(file)                                              # 데이터를 분석하여 토큰 단위로 생성하는 듯
        doc = PDFDocument(parser)                                             # 생성된 토큰들을 다시 문서 단위로 묶는 방식으로 추정
        rsrcmgr = PDFResourceManager()                                        # pdf resource manager 객체 생성
        device = TextConverter(rsrcmgr, output_String, laparams = LAParams()) # 위의 객체를 활용하여 text 형식으로 변환?
        
        # 변환한 내역을 페이지별로 구분하는 과정으로 추정 
        interpreter = PDFPageInterpreter(rsrcmgr, device)                   
        count = 0
        for page in PDFPage.create_pages(doc):
            count += 1
            #(텍스트화를 희망하는 페이지가 아니면 skip하도록 변형)
            if count < convertStartNo: 
                continue
            else:
                interpreter.process_page(page)
        return output_String.getvalue()    

#### 불필요한 공백 제거 후 문장별로 나누기(1차 정리)

In [3]:
# 희망 디렉터리 안에 pdf 파일 경로 list 만들기
list_filepath = glob.glob('./Data/*.pdf')
count = 0

df = pd.DataFrame(columns = ["내용"])
# 파일 경로를 하나씩 불러와 read_pdf 함수 호출
for filepath in list_filepath:

    text = read_pdf(filepath, 2)     # 첫 페이지는 개인정보 페이지이므로 생략
    textlist = text.split(". ")
    
    text_L = []
    for t in textlist:
        t = " ".join(t.split())
        text_L.append(t)
    df.loc[count] = [text_L]    
    count += 1

In [4]:
df.to_excel("./df_sample01.xlsx")

In [6]:
df_r = pd.read_excel("./df_sample01.xlsx", engine = "openpyxl", header=None)
df_r.head()

,0,1
0,NaN,내용
1,0.0,"['□ 나의 꿈과 끼, 인성(1,500자 이내, 띄어쓰기 제외) 접수번호 JC102..."
2,1.0,"['□ 나의 꿈과 끼, 인성(1,500자(띄어쓰기 제외) 이내) ○ 본인이 스스로 ..."
3,2.0,['']
4,3.0,"['□ 나의 꿈과 끼, 인성(1,500자 이내) 수험번호 1396 나는 세계적인 친..."
